In [ ]:
!pip install /kaggle/input/mypackages/timm-0.3.2-py3-none-any.whl
#!pip install /kaggle/input/mypackages/PyYAML-5.4.1-cp37-cp37m-manylinux1_x86_64.whl
!pip install /kaggle/input/mypackages/Pillow-7.2.0-cp37-cp37m-manylinux1_x86_64.whl
!pip install /kaggle/input/mypackages/thop-0.0.31.post2005241907-py3-none-any.whl

In [ ]:
import sys
sys.path.insert(0, "../input/yolov5")
sys.path.insert(1, "../input/mypackages")

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import argparse
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pathlib, sys, os, random, time
import numba, cv2, gc
import glob
from PIL import Image
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
from tqdm import tqdm
import albumentations as A
import rasterio
from rasterio.windows import Window
import torch
import torch.nn as nn
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torch.utils.data as D
import torchvision
from torchvision import transforms as T
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_img_size, check_requirements, check_imshow, non_max_suppression, apply_classifier, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

In [ ]:
DATA_PATH = '../input/hubmap-kidney-segmentation/'
p = pathlib.Path(DATA_PATH)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
import segmentation_models_pytorch as smp
print(DEVICE)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
EPOCHES = 50
BATCH_SIZE = 64
WINDOW = 1024
MIN_OVERLAP = 100
NEW_SIZE = 512
identity = rasterio.Affine(1, 0, 0, 0, 1, 0)
subm = {}
start_time = time.time()
conf_thres=0.25
iou_thres=0.5
classes=False
agnostic_nms =False

In [ ]:
def rle_numba(pixels):
    size = len(pixels)
    points = []
    if pixels[0] == 1: points.append(0)
    flag = True
    for i in range(1, size):
        if pixels[i] != pixels[i - 1]:
            if flag:
                points.append(i + 1)
                flag = False
            else:
                points.append(i + 1 - points[-1])
                flag = True
    if pixels[-1] == 1: points.append(size - points[-1] + 1)
    return points
def rle_numba_encode(image):
    pixels = image.flatten(order='F')
    points = rle_numba(pixels)
    return ' '.join(str(x) for x in points)
def make_grid(shape, window=256, min_overlap=32):
    """
        Return Array of size (N,4), where N - number of tiles,
        2nd axis represente slices: x1,x2,y1,y2
    """
    x, y = shape
    nx = x // (window - min_overlap) + 1
    x1 = np.linspace(0, x, num=nx, endpoint=False, dtype=np.int64)
    x1[-1] = x - window
    x2 = (x1 + window).clip(0, x)
    ny = y // (window - min_overlap) + 1
    y1 = np.linspace(0, y, num=ny, endpoint=False, dtype=np.int64)
    y1[-1] = y - window
    y2 = (y1 + window).clip(0, y)
    slices = np.zeros((nx, ny, 4), dtype=np.int64)

    for i in range(nx):
        for j in range(ny):
            slices[i, j] = x1[i], x2[i], y1[j], y2[j]
    return slices.reshape(nx * ny, 4)


In [ ]:
trfm_512 = T.Compose([
    T.ToPILImage(),
    T.Resize(512),
    T.ToTensor(),
    T.Normalize([0.64254668, 0.49338829, 0.68244302],
                [0.1927673, 0.25079806, 0.17813688]),
])
# trfm_384 = T.Compose([
#     T.ToPILImage(),
#     T.Resize(384),
#     T.ToTensor(),
#     T.Normalize([0.64254668, 0.49338829, 0.68244302],
#                 [0.1927673, 0.25079806, 0.17813688]),
# ])
yolo_trfm = T.Compose([
    T.ToPILImage(),
    T.Resize(NEW_SIZE),
    T.ToTensor(),
    # T.Normalize([0.64254668, 0.49338829, 0.68244302],
    #             [0.1927673,  0.25079806, 0.17813688]),
])

In [ ]:
segmodel_512_1 = smp.Unet(
    encoder_name="timm-efficientnet-b4",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7  se_resnext50_32x4d
    encoder_weights=None,  # use `imagenet` pretreined weights for encoder initialization
    in_channels=3,  # model input channels (1 for grayscale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
)
segmodel_512_1 = nn.DataParallel(segmodel_512_1)
segmodel_512_1.load_state_dict(torch.load("../input/hubmapmodel/out915_0.932_Unet_b4_v0428_fold1_epoch4_vdice0.949331.pth"))
segmodel_512_1.to(DEVICE);
segmodel_512_1.eval()

# segmodel_512_2 = smp.UnetPlusPlus(
#     encoder_name="timm-efficientnet-b2",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7  se_resnext50_32x4d
#     encoder_weights=None,  # use `imagenet` pretreined weights for encoder initialization
#     in_channels=3,  # model input channels (1 for grayscale images, 3 for RGB, etc.)
#     classes=1,  # model output channels (number of classes in your dataset)
# )
# segmodel_512_2 = nn.DataParallel(segmodel_512_2)
# segmodel_512_2.load_state_dict(torch.load("../input/hubmapmodel/out890_0.932_UNetPlusPluse_b2_v0429_fold0_epoch20_vdice0.949854.pth"))
# segmodel_512_2.to(DEVICE);
# segmodel_512_2.eval()


# segmodel_512_3 = smp.Unet(
#     encoder_name="timm-efficientnet-b2",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7  se_resnext50_32x4d
#     encoder_weights=None,  # use `imagenet` pretreined weights for encoder initialization
#     in_channels=3,  # model input channels (1 for grayscale images, 3 for RGB, etc.)
#     classes=1,  # model output channels (number of classes in your dataset)
# )
# segmodel_512_3 = nn.DataParallel(segmodel_512_3)
# segmodel_512_3.load_state_dict(torch.load("../input/hubmapmodel/out898_0.932_Unet_b2_v0426_fold4_epoch8_vdice0.955260.pth"))
# segmodel_512_3.to(DEVICE);
# segmodel_512_3.eval()

segmodel_512_4 = smp.MAnet(
    encoder_name="timm-efficientnet-b4",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7  se_resnext50_32x4d
    encoder_weights=None,  # use `imagenet` pretreined weights for encoder initialization
    in_channels=3,  # model input channels (1 for grayscale images, 3 for RGB, etc.)
    classes=1,  # model output channels (number of classes in your dataset)
)
segmodel_512_4 = nn.DataParallel(segmodel_512_4)
segmodel_512_4.load_state_dict(torch.load("../input/hubmapmodel/out918_0.931_MAnet_b4_v0502_fold1_epoch4_vdice0.954013.pth"))
segmodel_512_4.to(DEVICE);
segmodel_512_4.eval()


# segmodel_384_1 = smp.Unet(
#     encoder_name="timm-efficientnet-b3",  # choose encoder, e.g. mobilenet_v2 or efficientnet-b7  se_resnext50_32x4d
#     encoder_weights=None,  # use `imagenet` pretreined weights for encoder initialization
#     in_channels=3,  # model input channels (1 for grayscale images, 3 for RGB, etc.)
#     classes=1,  # model output channels (number of classes in your dataset)
# )
# segmodel_384_1.load_state_dict(torch.load("../input/hubmapnewmodel/timm_b3_all_384_fold_1_15.pth"))
# segmodel_384_1.to(DEVICE);
# segmodel_384_1.eval()



In [ ]:

subm = {}
''' yolo模型定义 '''
weights = '/kaggle/input/yolov5/runs/train/exp37/weights/best.pt'
imgsz = 512
# Initialize
device = select_device('0')
half = device.type != 'cpu'  # half precision only supported on CUDA
# Load model
model_yolo = attempt_load(weights, map_location=device)  # load FP32 model
stride = int(model_yolo.stride.max())  # model stride
imgsz = check_img_size(imgsz, s=stride)  # check img_size
if half:
    model_yolo.half()  # to FP16
# Run inference
if device.type != 'cpu':
    model_yolo(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model_yolo.parameters())))  # run once

print('1')
'''seg&yolo'''
for filei, filename in enumerate(p.glob('test/*.tiff')):
    dataset = rasterio.open(filename.as_posix(), transform = identity)
    slices = make_grid(dataset.shape, window=WINDOW, min_overlap=MIN_OVERLAP)
    print(filename)
    print(filei)
    check = 0
    yolotime = 0
    change_seg=0
#     aotu = 0


    preds = np.zeros(dataset.shape, dtype=np.uint8)

    for (x1, x2, y1, y2) in tqdm(slices):
        '''读入一张切片'''
        save_yolo = 0
#         print(check)
#         check +=1
        if dataset.count == 3:
            oimage = dataset.read([1, 2, 3],
                                 window=Window.from_slices((x1, x2), (y1, y2)))
            oimage = np.moveaxis(oimage, 0, -1)
        else:
            subdatasets = dataset.subdatasets
            #     if len(subdatasets) > 0:
            #  image = np.zeros(WINDOW, WINDOW, 3), dtype=np.uint8)
            oimage = np.zeros((1024, 1024, 3), dtype=np.uint8)
            for m, subdataset in enumerate(subdatasets, 0):
                with rasterio.open(subdataset) as layer:
                    oimage[:, :, m] = layer.read(1, window=Window.from_slices((x1, x2), (y1, y2)))

        img_save = oimage

        '''YOLO输入图片'''
        image_yolo = yolo_trfm(oimage)
        '''分割图片'''
        image_512 = trfm_512(oimage)
#         image_384 = trfm_384(oimage)

        '''seg分割'''
        with torch.no_grad():
            image_512 = image_512.to(DEVICE)[None]
#             image_384 = image_384.to(DEVICE)[None]

            score_512_1 = segmodel_512_1(image_512)[0][0]
#             score_512_2 = segmodel_512_2(image_512)[0][0]
#             score_512_3 = segmodel_512_3(image_512)[0][0]
            score_512_4 = segmodel_512_4(image_512)[0][0]    
#             score_384_1 = segmodel_384_1(image_384)[0][0]


            score_512_1 = score_512_1
            score_512_1_sigmoid = score_512_1.sigmoid().cpu().numpy()
            score_512_1_sigmoid = cv2.resize(score_512_1_sigmoid, (WINDOW, WINDOW))
            
#             score_512_2 = score_512_2
#             score_512_2_sigmoid = score_512_2.sigmoid().cpu().numpy()
#             score_512_2_sigmoid = cv2.resize(score_512_2_sigmoid, (WINDOW, WINDOW))
            
#             score_512_3 = score_512_3
#             score_512_3_sigmoid = score_512_3.sigmoid().cpu().numpy()
#             score_512_3_sigmoid = cv2.resize(score_512_3_sigmoid, (WINDOW, WINDOW))
            
            score_512_4 = score_512_4
            score_512_4_sigmoid = score_512_4.sigmoid().cpu().numpy()
            score_512_4_sigmoid = cv2.resize(score_512_4_sigmoid, (WINDOW, WINDOW))
            
#             score_384_1 = score_384_1
#             score_384_1_sigmoid = score_384_1.sigmoid().cpu().numpy()
#             score_384_1_sigmoid = cv2.resize(score_384_1_sigmoid, (WINDOW, WINDOW))


    
            score_mean = (score_512_1_sigmoid  + score_512_4_sigmoid ) / 2.0
#             score_mean = (score_512_1_sigmoid + score_512_2_sigmoid +
#                           score_512_3_sigmoid + score_512_4_sigmoid ) / 4.0

            
            '''填充空洞'''
            score_sigmoid = ( score_mean > 0.4).astype(np.uint8)
            if (score_sigmoid >0.5 ).mean() > 0.002:
                contours, hierarchy = cv2.findContours(score_sigmoid, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)#检测轮廓
                image_fill = np.zeros((1024,1024), dtype=np.uint8)
                cv2.fillPoly(score_sigmoid, contours, 1)
                area_ao = np.sum(np.greater(score_sigmoid, 0))#凹填充的面积计算
                for c in range(len(contours)):  # 凸包检测并填充区域 
                    points = cv2.convexHull(contours[c])
                    cv2.fillConvexPoly(image_fill, points, 1)  
                area_tu = np.sum(np.greater(image_fill, 0))  #凸填充面积计算
                area_rate = float(area_ao / area_tu)
            else :
                area_rate = 1
            preds[x1:x2, y1:y2] = (score_sigmoid > 0.5).astype(np.uint8)#凹填充的结果或者是没填充
            # 取出单张切片，等下用来和检测结果比较
            one_seg_pred = preds[x1:x2, y1:y2]

        '''单张切片分割结果有标注才跑检测。也可以反过来先检测有检测结果再分割，不过我估计效果很差'''
        if area_rate >0.6 and area_rate <0.79:
            yolotime +=1
            
            image_fill = (image_fill > 0.5).astype(np.uint8)   #凸填充的结果
            # new_seg_pred = one_seg_pred
            img_yolo = image_yolo.to(device)[None]
            img_yolo = img_yolo.half()
            # 没有batch_size的话则在最前面添加一个轴
            if img_yolo.ndimension() == 3:
                img_yolo = img_yolo.unsqueeze(0)
            # Inference
            """
                    前向传播 返回pred的shape是(1, num_boxes, 5+num_class)
                    h,w为传入网络图片的长和宽，注意dataset在检测时使用了矩形推理，所以这里h不一定等于w
                    num_boxes = h/32 * w/32 + h/16 * w/16 + h/8 * w/8
                    pred[..., 0:4]为预测框坐标
                    预测框坐标为xywh(中心点+宽长)格式
                    pred[..., 4]为objectness置信度
                    pred[..., 5:-1]为分类结果
            """
            augment =True
            pred_yolo = model_yolo(img_yolo, augment=augment)[0]
            # 进行NMS
            """
            pred:前向传播的输出
            conf_thres:置信度阈值
            iou_thres:iou阈值
            classes:是否只保留特定的类别
            agnostic:进行nms是否也去除不同类别之间的框
            经过nms之后，预测框格式：xywh-->xyxy(左上角右下角)
            pred是一个列表list[torch.tensor]，长度为batch_size
            """
            pred_yolo = non_max_suppression(pred_yolo, conf_thres, iou_thres, classes=classes,
                                            agnostic=agnostic_nms)

            # 依次导出每一个检测框
            for i1, det in enumerate(pred_yolo):
                # 判断是否有检测框。若无检测框，则不变
                if len(det):
                    # 检测框从512*512 到1024*1024
                    det[:, :4] = scale_coords(img_yolo.shape[2:], det[:, :4], oimage.shape).round()
                    box_x1 = int(det[0][0])
                    box_x2 = int(det[0][2])
                    box_y1 = int(det[0][1])
                    box_y2 = int(det[0][3])
                    
                    #  这里可以检查x1,x2,y1,y2的位置，检测之前已经凸填充了

                    # 画出检测框，
                    box = image_fill[ box_y1 :box_y2,box_x1:box_x2] 
                    # 计算检测框内的分割面积
                    seg_area=box.sum()
                    # 检测框面积
                    box_area = (box_x2-box_x1)*(box_y2-box_y1)
                    # 分割面积和监测面积的比值
                    area_rate = float(seg_area / box_area)
                    # 比值小于某值时，填充
                    if box_area > 5000 and area_rate<0.5 and area_rate>0.05:
                        change_seg +=1
 
                        # 填充方式:椭圆，
                        x = box.shape[0]
                        y = box.shape[1]

                        #调用cv2的包，先绘制rgb的椭圆，再转成单通道
                        fill = np.zeros((x,y,3),np.uint8)

                        fill_circle = cv2.circle(img=fill, center=(int(y / 2), int(x / 2)),
                                                 radius=min(int(x / 3), int(y / 3)),
                                                 color=(1, 1, 1), thickness=-1)
                        fill_one = cv2.cvtColor(fill_circle, cv2.COLOR_RGB2GRAY)

                        new_pred = fill_one+box
                        new_pred = (new_pred > 0.5).astype(np.uint8)

                        contours, hierarchy = cv2.findContours(new_pred, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)  # 检测轮廓
                        for c in range(len(contours)):  # 凸包检测并填充区域
                            points = cv2.convexHull(contours[c])
                            cv2.fillConvexPoly(new_pred, points, 1)  # 填充轮廓

                        image_fill[box_y1:box_y2, box_x1:box_x2] = (new_pred > 0.5).astype(np.uint8)


                # 将新填充结果保存到最大的切片preds
            preds[x1:x2, y1:y2] = image_fill  #有可能没有目标直接执行这一行，有可能只是凸填充的结果
       

    print(filename.stem,'yolo time:', yolotime )
    print(filename.stem,'change seg time:', change_seg )

        # break

    #生成csv
    subm[filei] = {'id': filename.stem, 'predicted': rle_numba_encode(preds)}

    del preds
    gc.collect();
    
print('time:', (time.time() - start_time) // 60, 'min', (time.time() - start_time) % 60, 's')
submission = pd.DataFrame.from_dict(subm, orient='index')
submission.to_csv('./submission.csv', index=False)

